# 02: Dynamic Tool Registration

This notebook provides a hands-on guide to dynamically registering tools with the Agent Gateway. The Agent Gateway allows you to extend the `ScientificWorkflowAgent`'s capabilities at runtime without restarting any services. We will cover registering and using various tool types:

*   Simple `stdio` scripts
*   Containerized `stdio` tools
*   The universal `docker mcp gateway run` adapter
*   HTTP-based tools

## Getting Started

Before running this notebook, ensure that the Docker services for Chapter 06 are up and running. You can start them by navigating to the `chapter-06-advanced-multi-agent-orchestration` directory and executing the lifecycle script:

```bash
./start-chapter-resources.sh
```

Also, ensure you have an SSH session with port forwarding for both OpenWebUI (`8902`) and the Agent Gateway (`8083`):

```bash
ssh -L 8902:localhost:8902 -L 8083:localhost:8083 user@remote-server
```

## 1. Dynamically Registering a `stdio` Tool (Python Script)

First, we will register a simple Python script (`sys_info_tool.py`) as a `stdio` tool. This script reads JSON from `stdin` and writes its result as JSON to `stdout`.

### Step 1: Review the Tool's Python Script

The `sys_info_tool.py` script is located at `src/agentic_framework_pkg/agent_gateway/tools/sys_info_tool.py`. It retrieves basic system information from the container it's running in.

### Step 2: Review the Tool's JSON Registration File

The JSON file (`stdio_sys_info_tool.json`) describes our new tool to the Agent Gateway. It specifies the tool's name, description, arguments schema, and how to invoke it (`stdio` protocol with a `command`). This file is located at `src/agentic_framework_pkg/agent_gateway/config/stdio_sys_info_tool.json`.

### Step 3: Register the New Tool

We will send the JSON configuration to the running `agent_gateway`'s `/v1/tools/register` endpoint using `curl`.

Open a **new local terminal** (not your SSH session) and run the following `curl` command:

```bash
curl -X POST http://localhost:8083/v1/tools/register \
-H "Content-Type: application/json" \
-d @docs/tutorial-branches/chapter-06-advanced-multi-agent-orchestration/src/agentic_framework_pkg/agent_gateway/cli/examples/stdio_sys_info_tool.json
```

You should see a success message like: `{"status":"success","message":"Tool 'get_system_information' registered successfully..."}`

### Step 4: Verify the Registration

Now, let's verify that the Agent Gateway recognizes the new tool by querying the `/tools/{model_id}` endpoint.


In [2]:
import requests
import json

AGENT_GATEWAY_URL = "http://localhost:8083"
MODEL_ID = "agentic-framework/scientific-agent-v1"

try:
    response = requests.get(f"{AGENT_GATEWAY_URL}/tools/{MODEL_ID}")
    response.raise_for_status()
    tools = response.json()
    # Filter to show only our newly registered tool for clarity
    sys_info_tool = next(([t for t in tools["tools"] if t["name"] == "get_system_information"]), None)
    if sys_info_tool:
        print(json.dumps(sys_info_tool, indent=2))
    else:
        print("Tool 'get_system_information' not found.")
except requests.exceptions.RequestException as e:
    print(f"Error connecting to Agent Gateway: {e}")
except json.JSONDecodeError:
    print(f"Error decoding JSON response: {response.text}")


Error connecting to Agent Gateway: HTTPConnectionPool(host='localhost', port=8083): Max retries exceeded with url: /tools/agentic-framework/scientific-agent-v1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffffa751f010>: Failed to establish a new connection: [Errno 111] Connection refused'))


### Step 5: Use the New Tool in a Chat Session

Now, you can interact with the agent via OpenWebUI and ask it to use the newly registered tool. Go to OpenWebUI at `http://localhost:8902` and start a new chat. Ask the agent a question that would cause it to use your new tool, for example:

`What is the system information of the server you are running on?`

The agent should respond with details about the operating system, architecture, Python version, and container hostname.

## 2. Progressing to a Containerized `stdio` Tool

The true power of the `stdio` protocol is its flexibility. The gateway doesn't care *what* it runs, only that it can execute the command and communicate via stdin/stdout. We can demonstrate this by packaging our simple Python script into its own standalone Docker container and registering it as a new tool.

### Step 1: Review the Dockerfile for the Tool

The `Dockerfile.sys_info_tool` is located at `src/agentic_framework_pkg/agent_gateway/tools/Dockerfile.sys_info_tool`. It packages the `sys_info_tool.py` script into a Docker image.

### Step 2: Build the Docker Image

On your **remote server** (via your SSH session), build the Docker image for the tool. This command must be run from the root of the `agentic_framework` project directory (e.g., `/Users/rigo160/Documents-Local/LocalWorkspace/MCP-Tool-Framework-v8-2025-06-03/agentic_framework/`).

```bash
docker build -t sys-info-tool:latest -f docs/tutorial-branches/chapter-06-advanced-multi-agent-orchestration/src/agentic_framework_pkg/agent_gateway/tools/Dockerfile.sys_info_tool docs/tutorial-branches/chapter-06-advanced-multi-agent-orchestration/src/agentic_framework_pkg/agent_gateway/tools/
```

### Step 3: Review the New JSON Registration File

The `stdio_sys_info_tool_docker.json` file is located at `src/agentic_framework_pkg/agent_gateway/cli/examples/stdio_sys_info_tool_docker.json`. The only significant change from the previous registration file is the `command` field, which now points to `docker run`:

```json
{
  "name": "get_system_information_docker",
  "description": "A DOCKERIZED tool that retrieves basic system information from its own container, such as OS type and architecture.",
  "args_schema": {},
  "invocation": {
    "protocol": "stdio",
    "connection": {
      "command": [
        "docker",
        "run",
        "--rm",
        "-i",
        "sys-info-tool:latest"
      ]
    }
  },
  "acl": {
    "global": true
  }
}
```

### Step 4: Register and Use the Dockerized Tool

Register the new tool using `curl` from your **local terminal**:

```bash
curl -X POST http://localhost:8083/v1/tools/register \
-H "Content-Type: application/json" \
-d @docs/tutorial-branches/chapter-06-advanced-multi-agent-orchestration/src/agentic_framework_pkg/agent_gateway/cli/examples/stdio_sys_info_tool_docker.json
```

Verify its registration using the Python code from Section 1, but search for `get_system_information_docker`. Then, in OpenWebUI, ask the agent:

`Use the DOCKERIZED tool to get the system information.`

The agent will now invoke the `docker run` command, and you will get the system information from the ephemeral `sys-info-tool` container.

## 3. Using the `docker mcp gateway run` Universal Adapter

This section demonstrates the most advanced and decoupled method for tool invocation: using a universal adapter for the `docker mcp gateway run` command. This pattern provides maximum flexibility, allowing you to add and modify underlying tools without ever changing the registration data in the `agent_gateway`.

### Step 1: Review the Gateway Adapter Script

The `gateway_run_adapter.py` script is located at `src/agentic_framework_pkg/agent_gateway/tools/gateway_run_adapter.py`. This simple Python script acts as a bridge: the `agent_gateway` will execute this script, which in turn executes the `docker mcp gateway run` command.

### Step 2: Review the Universal Tool Registration File

The `stdio_docker_mcp_gateway_run_example.json` file is located at `src/agentic_framework_pkg/agent_gateway/cli/examples/stdio_docker_mcp_gateway_run_example.json`. This file registers our universal adapter. Notice the `args_schema` fields: `mcp_tool_name` (the name of the *actual* tool to run) and `mcp_tool_args` (the arguments for that tool).

### Step 3: Register the Universal Adapter

Using your local, port-forwarded terminal, register this new universal tool:

```bash
curl -X POST http://localhost:8083/v1/tools/register \
-H "Content-Type: application/json" \
-d @docs/tutorial-branches/chapter-06-advanced-multi-agent-orchestration/src/agentic_framework_pkg/agent_gateway/cli/examples/stdio_docker_mcp_gateway_run_example.json
```

### Step 4: Use the Universal Adapter in a Chat Session

Now, you can invoke *any* tool known to the `docker mcp` environment through this single, unified interface. Go to OpenWebUI and ask the agent:

`Use the invoke_mcp_gateway_tool to run the get_system_information tool with no arguments.`

The agent will call `invoke_mcp_gateway_tool`, providing `mcp_tool_name="get_system_information"` and `mcp_tool_args={}`. The `agent_gateway` runs the adapter script, which then executes `docker mcp gateway run get_system_information`. The result is passed back to the agent, which formulates the final answer.

## 4. Registering and Using an HTTP-Based Tool

This section showcases how to register a tool that communicates with an external service over HTTP, allowing the agent to interact with existing microservices or public APIs.

### Step 1: Review the HTTP Tool Registration File

The `http_tool_example.json` file is located at `src/agentic_framework_pkg/agent_gateway/cli/examples/http_tool_example.json`. It defines a hypothetical `get_weather_conditions` tool that makes an HTTP `GET` request to `https://api.example.com/weather/current`.

### Step 2: Register the HTTP Tool

In your **local terminal** (with the SSH port forward active), run the `curl` command to register the tool:

```bash
curl -X POST http://localhost:8083/v1/tools/register \
-H "Content-Type: application/json" \
-d @docs/tutorial-branches/chapter-06-advanced-multi-agent-orchestration/src/agentic_framework_pkg/agent_gateway/cli/examples/http_tool_example.json
```

### Step 3: Verify and Use the Tool

Verify its registration using the Python code from Section 1, but search for `get_weather_conditions`. Then, in OpenWebUI, ask the agent:

`What is the weather like in Seattle?`

The agent will call the tool. While this specific example will fail (as the API is hypothetical), it demonstrates the mechanism for integrating real-world APIs.

This concludes the dynamic tool registration tutorial. In the next notebook, we will explore the `ScientificWorkflowAgent`'s RAG capabilities.
